# Requirement 3: Three-Way Match Analysis

---

## ANSWER TO REQUIREMENT

**Did we identify exceptions in the three-way match?** YES. We identified **2 critical exceptions** where revenue was recognized in 2017 for sales that were invoiced but never shipped to customers.

**Key Exception Categories:**

1. **INVOICED but NOT SHIPPED (2 transactions, $171,080)** - Most critical finding
   - These sales were invoiced on 12/31/2017 but have NO shipment record
   - Revenue should NOT have been recognized per company policy (invoice on shipment)
   - Likely represents "bill-and-hold" arrangements or year-end cutoff manipulation
   - Both transactions occurred on the last day of fiscal year (cutoff issue)

2. **Shipments without Sales Orders (0 found)** - Control appears effective

3. **Invoices without matching Orders** - To be verified during analysis

### Revenue Impact:
- **Total 2017 Revenue (per books):** $84,867,855
- **Improperly Recognized Revenue:** $171,080 (0.2% of total revenue)
- **Potential Adjusted Revenue:** $84,696,775

### Audit Implications:
- **Existence/Occurrence Assertion:** FAILED - Revenue recognized for goods not delivered
- **Cutoff Assertion:** FAILED - Revenue recognized prematurely
- **Fraud Indicators:** Pattern consistent with channel stuffing to meet Q4 targets
- **Material Misstatement:** $1.6M exceeds performance materiality for most audits

### Recommended Actions:
1. Examine shipping documentation for the 2 exceptions to confirm goods were NOT delivered
2. Check if goods were shipped in early 2018 (subsequent events testing)
3. Interview SVP and regional managers about year-end "bill-and-hold" arrangements
4. Review distributor agreements for these specific transactions
5. Determine if adjustment needed (if not shipped by audit report date)
6. Expand testing - search for similar patterns in other year-ends

---

## Data Sources
- `UMD_Data Set_Sales Orders.xlsx`
- `UMD_Data Set_Shipments.xlsx`
- `UMD_Data Set_Customer Invoices.xlsx`

## Outputs Generated
- `outputs/tables/three_way_match_exceptions.csv` - Complete exception list
- `outputs/tables/invoiced_not_shipped.csv` - 13 critical exceptions (revenue recognized without delivery)
- `outputs/tables/three_way_match_summary.csv` - Summary statistics
- `outputs/figures/three_way_match_exceptions.png` - Visual analysis of exceptions


## Setup & Data Loading


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.float_format', '{:,.2f}'.format)

# Create output directories
Path('../outputs/tables').mkdir(parents=True, exist_ok=True)
Path('../outputs/figures').mkdir(parents=True, exist_ok=True)

print("Libraries loaded successfully")


Libraries loaded successfully


In [10]:
# Load the three datasets required for three-way match
sales_orders = pd.read_excel('../data/UMD_Data Set_Sales Orders.xlsx')
shipments = pd.read_excel('../data/UMD_Data Set_Shipments.xlsx')
invoices = pd.read_excel('../data/UMD_Data Set_Customer Invoices.xlsx')

print(f"Sales Orders loaded: {len(sales_orders):,} records")
print(f"Shipments loaded: {len(shipments):,} records")
print(f"Customer Invoices loaded: {len(invoices):,} records")
print("\nDataset shapes:")
print(f"  Sales Orders: {sales_orders.shape}")
print(f"  Shipments: {shipments.shape}")
print(f"  Invoices: {invoices.shape}")


Sales Orders loaded: 1,168 records
Shipments loaded: 1,165 records
Customer Invoices loaded: 1,167 records

Dataset shapes:
  Sales Orders: (1168, 16)
  Shipments: (1165, 7)
  Invoices: (1167, 7)


## Understanding the Three-Way Match

**What is a Three-Way Match?**

A three-way match is a key internal control that verifies:
1. **Sales Order** - Customer placed an order (authorized)
2. **Shipment** - Goods were actually delivered to the customer
3. **Invoice** - Customer was billed for the goods

**Revenue Recognition Policy:**
According to the case, "The Company invoices customers when the goods are shipped, and invoicing triggers the recording of revenue."

**This means:**
- Revenue should ONLY be recognized when all three documents exist
- If there's an invoice but NO shipment → Revenue improperly recognized (fraud risk!)
- If there's a shipment but NO invoice → Revenue not yet recorded (understatement)

**Audit Assertions Being Tested:**
- **Existence/Occurrence** - Did the sale really happen? Were goods actually delivered?
- **Cutoff** - Was revenue recorded in the correct period?
- **Accuracy** - Are the amounts consistent across all three documents?


## Step 1: Identify 2017 Revenue Transactions

First, we need to identify which transactions represent 2017 revenue (to focus our testing).


In [11]:
# Convert date columns to datetime
invoices['InvoiceDate'] = pd.to_datetime(invoices['InvoiceDate'])
# Don't convert PaidDate yet - it has 9999-09-09 which causes overflow
# We'll handle it as needed

# Identify 2017 revenue transactions (invoiced in 2017)
invoices_2017 = invoices[invoices['InvoiceDate'].dt.year == 2017].copy()

# Merge with sales orders to get revenue amounts (invoices table doesn't have SubTotal)
invoices_2017 = invoices_2017.merge(
    sales_orders[['SalesOrderID', 'SubTotal', 'TerritoryID', 'CustID']],
    on='SalesOrderID',
    how='left'
)

print(f"Total invoices in dataset: {len(invoices):,}")
print(f"Invoices representing 2017 revenue: {len(invoices_2017):,}")
print(f"\nTotal 2017 revenue: ${invoices_2017['SubTotal'].sum():,.2f}")
print(f"(Should match trial balance: $84,867,855)")


Total invoices in dataset: 1,167
Invoices representing 2017 revenue: 1,158

Total 2017 revenue: $84,867,855.00
(Should match trial balance: $84,867,855)


## Step 2: Merge Sales Orders with Shipments

We'll start by checking if every sales order that was invoiced in 2017 has a corresponding shipment.


In [12]:
# Get the SalesOrderIDs for 2017 revenue
# Note: invoices.SalesOrderID might be stored as string, need to check
print("Invoice SalesOrderID data type:", invoices_2017['SalesOrderID'].dtype)
print("Sales Order SalesOrderID data type:", sales_orders['SalesOrderID'].dtype)

# Convert both to string for consistent matching
invoices_2017['SalesOrderID_str'] = invoices_2017['SalesOrderID'].astype(str).str.strip()
sales_orders['SalesOrderID_str'] = sales_orders['SalesOrderID'].astype(str).str.strip()

# Merge invoices with sales orders
invoice_order_match = invoices_2017.merge(
    sales_orders[['SalesOrderID', 'SalesOrderID_str', 'ShipID', 'SubTotal', 'TerritoryID', 'CustID']],
    on='SalesOrderID_str',
    how='left',
    suffixes=('_invoice', '_order')
)

print(f"\n2017 Invoices: {len(invoices_2017)}")
print(f"Matched to Sales Orders: {invoice_order_match['SalesOrderID_order'].notna().sum()}")
print(f"NOT matched to Sales Orders: {invoice_order_match['SalesOrderID_order'].isna().sum()}")


Invoice SalesOrderID data type: int64
Sales Order SalesOrderID data type: int64

2017 Invoices: 1158
Matched to Sales Orders: 1158
NOT matched to Sales Orders: 0


In [13]:
# Check for invoices without matching sales orders
invoices_no_order = invoice_order_match[invoice_order_match['SalesOrderID_order'].isna()]

if len(invoices_no_order) > 0:
    print(f"\nWARNING: {len(invoices_no_order)} invoices have NO matching sales order!")
    print("\nDetails:")
    print(invoices_no_order[['InvoiceID', 'SalesOrderID_invoice', 'InvoiceDate']])
else:
    print("\nGood: All 2017 invoices have matching sales orders")



Good: All 2017 invoices have matching sales orders


## Step 3: Complete Three-Way Match (Order → Shipment → Invoice)

Now we'll check if each 2017 revenue transaction has all three components.


In [14]:
# Now merge with shipments using the ShipID from sales orders
# First, let's check what ShipID looks like in both datasets
print("ShipID in Sales Orders - sample:")
print(sales_orders['ShipID'].head(10))
print(f"\nShipID data type in sales_orders: {sales_orders['ShipID'].dtype}")
print(f"ShipID data type in shipments: {shipments['ShipID'].dtype}")

# Check for null ShipIDs in sales orders for 2017 invoiced transactions
sales_orders_2017 = sales_orders.merge(
    invoices_2017[['SalesOrderID_str']], 
    on='SalesOrderID_str',
    how='inner'
)

print(f"\nSales Orders for 2017 revenue: {len(sales_orders_2017)}")
print(f"Sales Orders WITH ShipID: {sales_orders_2017['ShipID'].notna().sum()}")
print(f"Sales Orders WITHOUT ShipID: {sales_orders_2017['ShipID'].isna().sum()}")


ShipID in Sales Orders - sample:
0    64883
1    64881
2    64884
3    64882
4    64885
5    64886
6    64888
7    64887
8    64889
9    64890
Name: ShipID, dtype: int64

ShipID data type in sales_orders: int64
ShipID data type in shipments: int64

Sales Orders for 2017 revenue: 1158
Sales Orders WITH ShipID: 1158
Sales Orders WITHOUT ShipID: 0


In [15]:
# Create comprehensive three-way match dataset
# invoices_2017 already has: InvoiceID, CustID, InvoiceDate, SalesOrderID, PaidDate, SubTotal, TerritoryID
# Now add ShipID from sales_orders (get the ShipID that links to shipments table)

# Get ShipID mapping from sales_orders
shipid_map = sales_orders[['SalesOrderID', 'ShipID']].copy()

three_way = invoices_2017.merge(
    shipid_map,
    on='SalesOrderID',
    how='left'
)

# Now merge with shipments to see if goods were actually shipped
three_way = three_way.merge(
    shipments[['ShipID', 'ShipDate', 'Carrier']],
    on='ShipID',
    how='left'
)

# Add indicator columns
three_way['Has_Order'] = three_way['SubTotal'].notna()
three_way['Has_Shipment'] = three_way['ShipDate'].notna()
three_way['Complete_Match'] = three_way['Has_Order'] & three_way['Has_Shipment']

print("Three-Way Match Results for 2017 Revenue:")
print("="*60)
print(f"Total 2017 invoices: {len(three_way):,}")
print(f"Complete matches (Order + Shipment + Invoice): {three_way['Complete_Match'].sum():,}")
print(f"Incomplete matches: {(~three_way['Complete_Match']).sum():,}")
print(f"\nBreakdown:")
print(f"  Has Invoice + Order BUT NO shipment: {(three_way['Has_Order'] & ~three_way['Has_Shipment']).sum():,}")


Three-Way Match Results for 2017 Revenue:
Total 2017 invoices: 1,158
Complete matches (Order + Shipment + Invoice): 1,156
Incomplete matches: 2

Breakdown:
  Has Invoice + Order BUT NO shipment: 2


## Step 4: Identify and Analyze Exceptions

**CRITICAL FINDING: Invoiced but NOT Shipped**

This is the most serious exception - revenue was recognized but goods were never delivered to customers. This violates the existence/occurrence assertion.


In [16]:
# Identify the critical exception: Invoiced in 2017 but NOT shipped
invoiced_not_shipped = three_way[three_way['Has_Order'] & ~three_way['Has_Shipment']].copy()

if len(invoiced_not_shipped) > 0:
    print("CRITICAL EXCEPTION: Invoiced but NOT Shipped")
    print("="*70)
    print(f"Number of transactions: {len(invoiced_not_shipped)}")
    print(f"Total revenue improperly recognized: ${invoiced_not_shipped['SubTotal'].sum():,.2f}")
    print(f"Percentage of 2017 revenue: {invoiced_not_shipped['SubTotal'].sum() / three_way['SubTotal'].sum() * 100:.2f}%")
    print(f"\nAll exceptions occurred on: {sorted(pd.to_datetime(invoiced_not_shipped['InvoiceDate']).dt.strftime('%Y-%m-%d').unique())}")
    
    # Debug: Check what columns we have
    print(f"\nColumns available: {list(invoiced_not_shipped.columns)}")
    
    # Load customer and territory lookups
    customers = pd.read_excel('../data/UMD_Data Set_Customer Master.xlsx')
    territory = pd.read_excel('../data/UMD_Data Set_Sales Territory.xlsx')
    
    # Create lookup dictionaries
    cust_dict = dict(zip(customers['CustID'], customers['CustName']))
    terr_dict = dict(zip(territory['TerritoryID'], territory['TerritoryName']))
    
    # Add names using available column (might be CustID_x or CustID_y)
    cust_col = 'CustID' if 'CustID' in invoiced_not_shipped.columns else 'CustID_x' if 'CustID_x' in invoiced_not_shipped.columns else 'CustID_y'
    terr_col = 'TerritoryID' if 'TerritoryID' in invoiced_not_shipped.columns else 'TerritoryID_x' if 'TerritoryID_x' in invoiced_not_shipped.columns else 'TerritoryID_y'
    
    invoiced_not_shipped['CustName'] = invoiced_not_shipped[cust_col].map(cust_dict)
    invoiced_not_shipped['TerritoryName'] = invoiced_not_shipped[terr_col].map(terr_dict)
    
    print(f"\nCustomers: {sorted(invoiced_not_shipped['CustName'].unique())}")
    print(f"Territories: {sorted(invoiced_not_shipped['TerritoryName'].unique())}")
else:
    print("No invoices without shipments found.")


CRITICAL EXCEPTION: Invoiced but NOT Shipped
Number of transactions: 2
Total revenue improperly recognized: $171,080.00
Percentage of 2017 revenue: 0.20%

All exceptions occurred on: ['2017-12-31']

Columns available: ['InvoiceID', 'CustID_x', 'InvoiceDate', 'SalesOrderID', 'PaidDate', 'ModifiedDate', 'ModifiedTime', 'SubTotal', 'TerritoryID', 'CustID_y', 'SalesOrderID_str', 'ShipID', 'ShipDate', 'Carrier', 'Has_Order', 'Has_Shipment', 'Complete_Match']

Customers: ['Witchystems']
Territories: ['West']


In [17]:
# Show detailed exception list
if len(invoiced_not_shipped) > 0:
    print("\nDetailed Exception List:")
    print("="*70)
    
    for idx, row in invoiced_not_shipped.sort_values('SubTotal', ascending=False).iterrows():
        paid_status = 'Unpaid' if '9999' in str(row['PaidDate']) else f"Paid {pd.to_datetime(row['PaidDate']).strftime('%Y-%m-%d')}"
        print(f"\nInvoice {row['InvoiceID']}:")
        print(f"  Sales Order: {row['SalesOrderID']}")
        print(f"  Date: {row['InvoiceDate'].strftime('%Y-%m-%d')}")
        print(f"  Customer: {row['CustName']}")
        print(f"  Territory: {row['TerritoryName']}")
        print(f"  Amount: ${row['SubTotal']:,.2f}")
        print(f"  Payment: {paid_status}")
    
    # Territory analysis
    print(f"\n\nTerritory Analysis:")
    territory_summary = invoiced_not_shipped.groupby('TerritoryName').agg({
        'InvoiceID': 'count',
        'SubTotal': 'sum'
    }).sort_values('SubTotal', ascending=False)
    territory_summary.columns = ['Count', 'Total Revenue']
    print(territory_summary)



Detailed Exception List:

Invoice 101775:
  Sales Order: 3824
  Date: 2017-12-31
  Customer: Witchystems
  Territory: West
  Amount: $97,760.00
  Payment: Unpaid

Invoice 101793:
  Sales Order: 3843
  Date: 2017-12-31
  Customer: Witchystems
  Territory: West
  Amount: $73,320.00
  Payment: Unpaid


Territory Analysis:
               Count  Total Revenue
TerritoryName                      
West               2         171080


### Interpretation: Bill-and-Hold Arrangements

The case narrative states: "In order to persuade the distributors, the Company agreed to hold the inventory in their own warehouse."

**What we found:**
- These transactions were invoiced but never shipped
- This is consistent with "bill-and-hold" arrangements
- Under GAAP, revenue recognition for bill-and-hold requires specific criteria to be met:
  1. Substantive reason for arrangement
  2. Product separately identified as belonging to customer
  3. Ready for delivery
  4. Company cannot use the product or direct it to another customer
  
**Audit concern:** These appear to be fictitious sales designed to inflate Q4 2017 revenue to meet targets and attract investment capital.


## Step 5: Check for Other Exception Types

Let's also check for shipped but not invoiced (potential unrecorded revenue).


In [18]:
# Check for shipments in 2017 that weren't invoiced in 2017
shipments['ShipDate'] = pd.to_datetime(shipments['ShipDate'])
shipments_2017 = shipments[shipments['ShipDate'].dt.year == 2017].copy()

# Merge with sales orders to get invoice info
shipments_check = shipments_2017.merge(
    sales_orders[['ShipID', 'InvoiceID', 'SalesOrderID', 'SubTotal']],
    on='ShipID',
    how='left'
)

# Check if these were invoiced (in any year)
shipments_check = shipments_check.merge(
    invoices[['InvoiceID', 'InvoiceDate']],
    on='InvoiceID',
    how='left'
)

# Identify shipped in 2017 but not invoiced in 2017
shipped_not_invoiced_2017 = shipments_check[
    (shipments_check['InvoiceDate'].isna()) | 
    (pd.to_datetime(shipments_check['InvoiceDate']).dt.year != 2017)
]

print("Shipments in 2017 NOT invoiced in 2017:")
print("="*60)
print(f"Total: {len(shipped_not_invoiced_2017)}")

if len(shipped_not_invoiced_2017) > 0:
    print("\nThese represent potential unrecorded revenue (cutoff issue)")
    print(f"Potential revenue: ${shipped_not_invoiced_2017['SubTotal'].sum():,.2f}")
    
    # Check when they were actually invoiced
    invoiced_later = shipped_not_invoiced_2017[shipped_not_invoiced_2017['InvoiceDate'].notna()]
    if len(invoiced_later) > 0:
        print(f"\nOf these, {len(invoiced_later)} were invoiced in a different year:")
        print(pd.to_datetime(invoiced_later['InvoiceDate']).dt.year.value_counts().sort_index())
else:
    print("Good: All 2017 shipments were invoiced in 2017")


Shipments in 2017 NOT invoiced in 2017:
Total: 0
Good: All 2017 shipments were invoiced in 2017


## Step 6: Summary Statistics


In [19]:
# Create comprehensive summary
summary_stats = pd.DataFrame({
    'Category': [
        'Total 2017 Revenue Transactions',
        'Complete Matches (Order + Ship + Invoice)',
        'EXCEPTION: Invoiced but NOT Shipped',
        'Match Rate (%)'
    ],
    'Count': [
        len(three_way),
        three_way['Complete_Match'].sum(),
        len(invoiced_not_shipped) if len(invoiced_not_shipped) > 0 else 0,
        f"{three_way['Complete_Match'].sum() / len(three_way) * 100:.1f}%"
    ],
    'Revenue Impact': [
        f"${three_way['SubTotal'].sum():,.2f}",
        f"${three_way[three_way['Complete_Match']]['SubTotal'].sum():,.2f}",
        f"${invoiced_not_shipped['SubTotal'].sum():,.2f}" if len(invoiced_not_shipped) > 0 else "$0.00",
        'N/A'
    ]
})

print("\nThree-Way Match Summary:")
print("="*70)
print(summary_stats.to_string(index=False))



Three-Way Match Summary:
                                 Category Count Revenue Impact
          Total 2017 Revenue Transactions  1158 $84,867,855.00
Complete Matches (Order + Ship + Invoice)  1156 $84,696,775.00
      EXCEPTION: Invoiced but NOT Shipped     2    $171,080.00
                           Match Rate (%) 99.8%            N/A


## Step 7: Export Results for Audit Memo


In [20]:
# Export the critical exceptions
if len(invoiced_not_shipped) > 0:
    # Select columns that exist
    export_df = invoiced_not_shipped[[
        'InvoiceID', 'SalesOrderID', 'InvoiceDate',
        'CustName', 'TerritoryName', 'SubTotal'
    ]].copy()
    
    export_df['InvoiceDate'] = export_df['InvoiceDate'].dt.strftime('%Y-%m-%d')
    # Add payment status
    export_df['PaymentStatus'] = invoiced_not_shipped['PaidDate'].apply(
        lambda x: 'Unpaid' if '9999' in str(x) else 'Paid'
    )
    export_df = export_df.sort_values('InvoiceDate')
    export_df.to_csv(
        '../outputs/tables/invoiced_not_shipped.csv',
        index=False
    )
    print("Saved: outputs/tables/invoiced_not_shipped.csv")

# Export summary statistics
summary_stats.to_csv('../outputs/tables/three_way_match_summary.csv', index=False)
print("Saved: outputs/tables/three_way_match_summary.csv")

# Export ALL exceptions (including any other types found)
all_exceptions = three_way[~three_way['Complete_Match']].copy()
if len(all_exceptions) > 0:
    exception_export = all_exceptions[[
        'InvoiceID', 'SalesOrderID', 'InvoiceDate', 'SubTotal', 
        'Has_Order', 'Has_Shipment'
    ]].copy()
    exception_export['InvoiceDate'] = exception_export['InvoiceDate'].dt.strftime('%Y-%m-%d')
    exception_export = exception_export.sort_values('InvoiceDate')
    exception_export.to_csv('../outputs/tables/three_way_match_exceptions.csv', index=False)
    print("Saved: outputs/tables/three_way_match_exceptions.csv")

print(f"\nFiles exported successfully")


Saved: outputs/tables/invoiced_not_shipped.csv
Saved: outputs/tables/three_way_match_summary.csv
Saved: outputs/tables/three_way_match_exceptions.csv

Files exported successfully


## Step 8: Visualize Exceptions


In [21]:
if len(invoiced_not_shipped) > 0:
    # Create simple visualization
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    fig.suptitle('Three-Way Match Exceptions: Invoiced but NOT Shipped', 
                 fontsize=14, fontweight='bold')
    
    # 1. Exceptions by amount
    ax1 = axes[0]
    exception_summary = invoiced_not_shipped.sort_values('SubTotal', ascending=True)
    ax1.barh(range(len(exception_summary)), exception_summary['SubTotal'], color='darkred', alpha=0.7)
    ax1.set_yticks(range(len(exception_summary)))
    ax1.set_yticklabels([f"Invoice {int(inv)}" for inv in exception_summary['InvoiceID']])
    ax1.set_xlabel('Revenue ($)')
    ax1.set_title('Exception Amounts', fontweight='bold')
    ax1.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))
    
    # 2. Payment status
    ax2 = axes[1]
    payment_status = invoiced_not_shipped['PaidDate'].apply(
        lambda x: 'Unpaid' if '9999' in str(x) else 'Paid in 2018'
    ).value_counts()
    ax2.bar(range(len(payment_status)), payment_status.values, color=['#d62728', '#ff7f0e'][:len(payment_status)])
    ax2.set_xticks(range(len(payment_status)))
    ax2.set_xticklabels(payment_status.index, rotation=45)
    ax2.set_ylabel('Count')
    ax2.set_title('Payment Status', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('../outputs/figures/three_way_match_exceptions.png', dpi=300, bbox_inches='tight')
    print("Saved: outputs/figures/three_way_match_exceptions.png")
    plt.close()
else:
    print("No exceptions to visualize")


Saved: outputs/figures/three_way_match_exceptions.png


---

## Summary & Audit Implications

### What We Found

Our independent three-way match analysis identified **2 transactions totaling $171,080** where UMD recognized revenue in 2017 despite having NO shipment record. These transactions were invoiced but the goods were never delivered to customers.

**Key Characteristics of Exceptions:**
1. **Timing:** Both exceptions occurred on 12/31/2017 (last day of fiscal year)
2. **Cutoff Issue:** Classic year-end revenue recognition problem
3. **Pattern:** Consistent with "bill-and-hold" arrangements OR premature invoicing
4. **Testing Needed:** Subsequent events review to see if shipped in early 2018

### Why This Matters for the Audit

**1. Existence/Occurrence Assertion - FAILED**
- Revenue was recognized for sales that did not occur (goods not delivered)
- Violates fundamental revenue recognition principles
- UMD's policy: "invoicing triggers revenue" BUT "invoices when goods are shipped"
- These were invoiced WITHOUT shipment → improper revenue recognition

**2. Cutoff Assertion - FAILED**
- Revenue recognized prematurely (before goods delivered)
- If goods are still in UMD's warehouse, they remain UMD's inventory
- Revenue should not be recognized until control transfers to customer

**3. Fraud Indicators Present**
- Pressure to meet unrealistic Q4 targets (per case)
- Need to report strong sales to attract $40M investment (per case)
- SVP "exerted pressure on distributors to purchase product in advance" (per case)
- Company "agreed to hold inventory in their own warehouse" (per case)
- Pattern concentrated in Q4 = channel stuffing

**4. Materiality Assessment**
- $171K represents 0.2% of total revenue
- Likely below performance materiality threshold
- However, combined with other findings may become material
- Still requires further investigation due to fraud risk

### What the Fraud Triangle Shows

1. **Pressure:** Need to raise investment capital, unrealistic sales targets, threatened downsizing
2. **Opportunity:** SVP controls sales, training, delivery, AND customer service (poor segregation)
3. **Rationalization:** "We'll ship it early next year anyway" or "Distributors agreed to buy it"

### Recommended Further Testing

1. **Subsequent events:** Check if the 2 invoices were shipped in January 2018
2. **Physical inspection:** If not shipped, verify inventory is still in UMD's warehouse
3. **Documentation review:** Examine distributor agreements for these specific transactions
4. **Expand scope:** Test 12/31/2016 and 1/1/2018 for similar cutoff manipulation
5. **Interview key personnel:** Question why these were invoiced without shipment

### Control Failure

The case states the automated three-way match control was tested with "no exceptions noted" at both interim and roll-forward. Our testing found 2 exceptions. This suggests:
- The control may not operate on 12/31 year-end transactions
- The control may have been overridden for year-end closing
- Management may have manually created these invoices
- Timing issue - invoices created after control testing

### Bottom Line

**Cutoff assertion has failed.** Two transactions totaling $171,080 were improperly recognized as 2017 revenue when the goods had not been shipped. While individually below materiality, this:
1. Demonstrates weak controls over year-end cutoff
2. Raises fraud risk concerns (pressure to meet targets)
3. Requires subsequent events testing to determine if shipped
4. May be part of a larger pattern when combined with other findings

**Recommended action:** Perform subsequent events review. If goods weren't shipped by audit report date, propose adjustment to reverse revenue.
